# Adam vs. Grad Clip

*Jun 30, 2023*

----------------------------------

I was playing with [DreamBooth](https://github.com/TheLastBen/fast-stable-diffusion) one day. In case you missed it, it's a tool which fine-tunes Stable Diffusion text-to-image model on your photos, so you can then create beautiful portraits. And I believe it is truly a *"future's future future"* except one thing: training time. To create a custom model you need to wait for about 40 minutes on a Tesla T4 which is an exact opposite of fun.  

So I started thinking on how to speed up the fine-tuning, ideally without interfering with the training procedure too much and preserve image quality as much as possible. I ran a training code through a `line_profiler` to find out that it is pretty well optimized from the start. But not perfectly. Besides some other things a particular [line](https://github.com/TheLastBen/diffusers/blob/7b633fd9c441fc1b0e6cb75524cd568243523b1c/examples/dreambooth/train_dreambooth.py#L723) (it's still there) caught my eye:
```
Line   №  Hits        Time   Per Hit  % Time  Line Contents
==============================================================
     723   100  41914431.8  419144.3    10.1  accelerator.clip_grad_norm_(params_to_clip, args.max_grad_norm)
```
A substantial chunk of training time (10%) was spent on clipping gradient norms, and I had a great desire to remove it. So my first idea was to check if the gradient ever gets clipped, and if it doesn't then this line can be safely removed. However I have found exactly the opposite: the gradient was always getting clipped, because its raw norm was ~10<sup>3</sup> while the threshold was set to 1. But does it still make any difference?

# Adam Refresher

There's plenty of information about Adam optimizer on the [Internet](https://letmegooglethat.com/?q=how+Adam+optimizer+works) including the original [paper](https://arxiv.org/abs/1412.6980). So here's a quick recap. 

A formula for the weight update in the Adam algorithm is:

$$ m_t = \frac{1}{1 - \beta_1^t} \Bigl( \beta_1 m_{t-1} + \left( 1 - \beta_1 \right) g_t \Bigr) $$
$$ v_t = \frac{1}{1 - \beta_2^t} \Bigl( \beta_2 v_{t-1} + \left( 1 - \beta_2 \right) g_t^2 \Bigr) $$
$$ \Delta w_t = -\lambda \frac{m_t}{\sqrt{v_t + \varepsilon}} $$

We are interested in the $v_t$ part of the formula. Here $v_t$ is sort of an element-wise variance of the gradient $g_t$. Dividing by $v_t$ we force weight updates $\Delta w_t$ to have a fixed variance with some EMA fluctuations. 

# What's Up with the Norm?

Recall how many parameters do your models typically have. I bet it would be at least millions but mostly closer to billions nowadays. So our gradient vector has **huge** dimensionality and we take a norm of it. But according to the concentration of measure theorem this norm converges to a constant as dimensionality grows (at least locally, where gradient distribution can be seen fixed). And it is clear that scaling the gradient by a constant under Adam won't change the weight update, since gradients are being renormalized anyway. But for the sake of completeness let's show this explicitly:

Let's first unravel the recursion in the Adam's formulas:

$$ m_t = \frac{1 - \beta_1}{1 - \beta_1^t} \sum_{i=1}^{t} g_t \beta_1^{t-i} = \sum_{i=1}^{t} A_{ti} g_i $$
$$ v_t = \frac{1 - \beta_2}{1 - \beta_2^t} \sum_{i=1}^{t} g_t^2 \beta_2^{t-i} = \sum_{i=1}^{t} B_{ti} g_i^2 $$

Here $A_{ti}$ and $B_{ti}$ are just weights, independent of $g_t$. Let's call $\hat{g}_t = \dfrac{g_t}{\|g_t\|}$. As the norm is approximately constant, it can be written as $\hat{g}_t \approx c \cdot g_t$. Let's plug it into EMA expressions:

$$ \hat{m}_t = \sum_{i=1}^{t} A_{ti} \hat{g}_i = c \cdot \sum_{i=1}^{t} A_{ti} g_i = c \cdot m_t$$
$$ \hat{v}_t = \sum_{i=1}^{t} B_{ti} \hat{g}_i^2 = c^2 \cdot \sum_{i=1}^{t} B_{ti} g_i = c^2 \cdot v_t$$

If we now plug it into weight update formula, the constant $c$ just cancels out:

$$ \frac{\hat{m}_t}{\sqrt{\hat{v}_t + \varepsilon}} \approx \frac{c m_t}{\sqrt{c^2 v_t + \varepsilon}} = \frac{m_t}{\sqrt{v_t + \varepsilon}} $$

Couple of notes on when this is more likely to be true. Our logic is based on the assumption that gradient distribution doesn't change a lot and individual components are independent. This corresponds to the fine-tuning regime, where the weight vector is near the minimum, so the consequitive gradients are less correlated and their distribution doesn't drift much. And this is exactly the case for the Dreambooth fine-tuning! Also testing this hypothesis was very simple: just comment the corresponding line of code and compare the results. And if you do that, you'll see that the quality of generated images doesn't change while you shave off 10% of execution time. 

# Conclusion

Gradient norm clipping is a kinda costly operation, so don't add it to your model unless you really have to (numerical issues and whatnot). Adam will take care of variance scaling anyway. 

Have a good night.